In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Python version: 3.6

import os
import copy
import time
import numpy as np
from tqdm import tqdm

# from options import args_parser
from utils import get_dataset, average_weights, exp_details

import multiprocessing
from update import update_network_weight, get_model_weight

from training import client_train, centralized_training
from update import ASRLocalUpdate

def FL_training_rounds(args, model_in_path_root, model_out_path, train_dataset, test_dataset):
    train_loss = []                                                                 # list for training loss
    global_weights = None                                                           # initial global_weights

    
    for epoch in tqdm(range(args.epochs)):                                          # train for given global rounds
        print(f'\n | Global Training Round : {epoch+1} |\n')                        # print current round

        m = max(int(args.frac * args.num_users), 1)                                 # num of clients to train, min:1
        idxs_users = np.random.choice(range(args.num_users), m, replace=False)      # select by client_id
        pool = multiprocessing.Pool(processes=m)

        if args.STAGE == 0:                                                         # train ASR
            local_weights_en = []                                                   # weight list for ASR encoder
            local_weights_de = []                                                   # weight list for ASR decoder
        else:                                                                       # train AD classifier or toggling network
            local_weights = []                                                      # only 1 weight list needed
        local_losses = []                                                           # losses of training clients of this round

        try:
            if (epoch == 0) and (args.STAGE == 2):                                  # start from global model to train toggling network
                global_weights = get_model_weight(args=args, source_path=model_out_path + "_global/final/", network="toggling_network")
                                                                                    # local ASR and AD with global toggling network
                                                                                    # get toggling_network weights from model in model_out_path + "_global/final/"
            final_result = pool.starmap_async(client_train, [(args, model_in_path_root, model_out_path, train_dataset, test_dataset, idx,
                                                                  epoch, global_weights) for idx in idxs_users])
                                                                                    # train from model in model_in_path 
                                                                                    #                                 + "_global/final/", when stage=0
                                                                                    #                                 + "_client" + str(idx) + "_round" + str(args.epochs-1) + "/final/", o.w.
                                                                                    # or model in last round
                                                                                    # final result in model_out_path + "_client" + str(client_id) + "_round" + str(global_round)
        except Exception as e:
            print(f"An error occurred while using starmap_sync to run client_train: {str(e)}")
        
        finally:
            final_result.wait()                                                     # wait for all clients end
            results = final_result.get()                                            # get results
        
        for idx in range(len(results)):                                             # for each participated clients
            w, loss = results[idx]                                                  # function client_train returns w & loss
            if args.STAGE == 0:                                                     # train ASR
                local_weights_en.append(copy.deepcopy(w[0]))                        # save encoder weight for this client
                local_weights_de.append(copy.deepcopy(w[1]))                        # save decoder weight for this client
            else:                                                                   # train AD classifier or toggling network
                local_weights.append(copy.deepcopy(w))
            local_losses.append(loss)

        # aggregate weights
        if args.STAGE == 0:                                                         # train ASR
            global_weights = [average_weights(local_weights_en), average_weights(local_weights_de)]
        else:                                                                       # train AD classifier or toggling network
            global_weights = average_weights(local_weights)

        loss_avg = sum(local_losses) / len(local_losses)                            # average losses from participated client
        train_loss.append(loss_avg)                                                 # save loss for this round
    return global_weights

# FL stage 1: ASR & AD Classifier
def stage1_training(args, train_dataset, test_dataset):
    local_epoch = args.local_ep                                                     # save given number of local epoch
    ##########################################################
    # Centralized Training: train global ASR & AD Classifier #
    ##########################################################
    """
    args.local_ep = args.global_ep                                                  # use number of global epoch for global model
    args.STAGE = 0                                                                  # train ASR first
    centralized_training(args=args, model_in_path=args.pretrain_name, model_out_path=args.model_out_path+"_finetune", 
                         train_dataset=train_dataset, test_dataset=test_dataset, epoch=0)
                                                                                    # train from pretrain, final result in args.model_out_path + "_finetune" + "_global/final"
    args.STAGE = 1                                                                  # then train AD classifier
    centralized_training(args=args, model_in_path=args.model_out_path+"_finetune_global/final/", 
                         model_out_path=args.model_out_path, train_dataset=train_dataset, test_dataset=test_dataset, epoch=0)
                                                                                    # train from final result from last line, final result in args.model_out_path + "_global/final"
    """
    ##########################################################
    # FL: train local ASR & AD Classifier federally          #
    ##########################################################
    args.local_ep = local_epoch                                                     # use the given number of local epoch
    args.STAGE = 0                                                                  # train ASR first
    global_weights = FL_training_rounds(args=args, model_in_path_root=args.model_out_path, model_out_path=args.model_out_path+"_finetune",
                                        train_dataset=train_dataset, test_dataset=test_dataset)

    # update global model
    model = update_network_weight(args=args, source_path=args.model_out_path+"_global/final/", target_weight=global_weights, network="ASR") 
                                                                                    # update ASR in source_path with given weights
    model.save_pretrained(args.model_out_path+"_FLASR_global/final")
    
    args.STAGE = 1                                                                  # then train AD classifier
    global_weights = FL_training_rounds(args=args, model_in_path_root=args.model_out_path+"_finetune", model_out_path=args.model_out_path,
                                        train_dataset=train_dataset, test_dataset=test_dataset)

    # update global model
    model = update_network_weight(args=args, source_path=args.model_out_path+"_FLASR_global/final", target_weight=global_weights, network="AD")
                                                                                    # update AD classifier in source_path with given weights
    model.save_pretrained(args.model_out_path+"_FLAD_global/final")
    
    
# FL stage 2: Toggling Network
def stage2_training(args, train_dataset, test_dataset):
    local_epoch = args.local_ep                                                     # save given number of local epoch
    ##########################################################
    # Centralized Training: train global Toggling Network    #
    ##########################################################
    """
    args.local_ep = args.global_ep                                                  # use number of global epoch for global model
    centralized_training(args=args, model_in_path=args.model_in_path + "_FLAD_global/final/", model_out_path=args.model_out_path, 
                         train_dataset=train_dataset, test_dataset=test_dataset, epoch=0)
                                                                                    # train from model_in_path + "_FLAD_global/final/" (aggregated ASR & AD)
                                                                                    # final result in args.model_out_path + "_global/final"
    """
    ##########################################################
    # FL: train local Toggling Network federally             #
    ##########################################################
    args.local_ep = local_epoch                                                     # use the given number of local epoch
    global_weights = FL_training_rounds(args=args, model_in_path_root=args.model_in_path, model_out_path=args.model_out_path,
                                        train_dataset=train_dataset, test_dataset=test_dataset)
    # update global model
    model = update_network_weight(args=args, source_path=args.model_out_path+"_global/final", target_weight=global_weights, network="toggling_network")
                                                                                    # update toggling_network in source_path with given weights
    model.save_pretrained(args.model_out_path+"_final_global/final")

def extract_emb(args, train_dataset, test_dataset):
    if args.client_id == "public":
        idx = "public"
    else:
        idx = int(args.client_id)
    local_model = ASRLocalUpdate(args=args, dataset=train_dataset, global_test_dataset=test_dataset, 
                                 client_id=idx, model_in_path=args.model_in_path, model_out_path=None)
                                                                                      # initial dataset of current client
    local_model.extract_embs()
                                                                                      # from model_in_path model, update certain part using given weight
# if __name__ == '__main__':
print("I'm here")
start_time = time.time()

# define paths
path_project = os.path.abspath('..')

# args = args_parser()  
import argparse                                                          # get configuration
parser = argparse.ArgumentParser()
# federated arguments (Notation for the arguments followed from paper)
parser.add_argument('--epochs', type=int, default=2,
                    help="number of rounds of training")
parser.add_argument('--num_users', type=int, default=2,
                    help="number of users: K")
parser.add_argument('--frac', type=float, default=1.0,
                    help='the fraction of clients: C')
parser.add_argument('--local_ep', type=int, default=5,
                    help="the number of local epochs: E")
# model arguments
parser.add_argument('--model', type=str, default='data2vec', help='model name')
parser.add_argument('--dataset', type=str, default='adress', help="name \
                    of dataset")
parser.add_argument('--gpu', default=None, help="To use cuda, set \
                    to a specific GPU ID. Default set to use CPU.")
# additional arguments
parser.add_argument('--pretrain_name', type=str, default='facebook/data2vec-audio-large-960h', help="str used to load pretrain model")

parser.add_argument('-lam', '--LAMBDA', type=float, default=0.5, help="Lambda for GRL")
parser.add_argument('-st', '--STAGE', type=int, default=1, help="Current training stage")
parser.add_argument('-fl_st', '--FL_STAGE', type=int, default=1, help="Current FL training stage")
parser.add_argument('-GRL', '--GRL', action='store_true', default=False, help="True: GRL")
parser.add_argument('-model_in', '--model_in_path', type=str, default="/home/FedASR/dacs/federated/save/data2vec-audio-large-960h_new1_recall_client0_round1/", help="Where the global model is saved")
parser.add_argument('-model_out', '--model_out_path', type=str, default="/home/FedASR/dacs/federated/save/data2vec-audio-large-960h_new1_recall", help="Where to save the model")
parser.add_argument('-log', '--log_path', type=str, default="wav2vec2-base-960h_linear_GRL.txt", help="name for the txt file")
parser.add_argument('-csv', '--csv_path', type=str, default="wav2vec2-base-960h_GRL_0.5", help="name for the csv file")
# 2023/01/08: loss type
parser.add_argument('-ad_loss', '--AD_loss', type=str, default="cel", help="loss to use for AD classifier")
# 2023/01/18: ckpt
parser.add_argument('-ckpt', '--checkpoint', type=str, default=None, help="path to checkpoint")
# 2023/02/13: TOGGLE_RATIO
parser.add_argument('-toggle_rt', '--TOGGLE_RATIO', type=float, default=0, help="To toggle more or less")
# 2023/02/15: GS_TAU, loss weight
parser.add_argument('-gs_tau', '--GS_TAU', type=float, default=1, help="Tau for gumbel_softmax")
parser.add_argument('-w_loss', '--W_LOSS', type=float, default=None, nargs='+', help="weight for HC and AD")
# 2023/04/20
parser.add_argument('-EXTRACT', '--EXTRACT', action='store_true', default=False, help="True: extract embs")
parser.add_argument('-client_id', '--client_id', type=str, default="public", help="client_id: public, 0, or 1")
# 2023/04/24
parser.add_argument('--global_ep', type=int, default=30, help="number for global model")

args = parser.parse_args(args=[])
exp_details(args)                                                               # print out details based on configuration

train_dataset, test_dataset = get_dataset(args)                                 # get dataset
# _, test_dataset = get_dataset(args)
# train_dataset=test_dataset #先這樣Debug
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

multiprocessing.set_start_method('spawn', force=True)
if args.EXTRACT != True:                                                        # Training
    if args.FL_STAGE == 1:
        print("| Start FL Training Stage 1|")
        stage1_training(args, train_dataset, test_dataset)                      # Train ASR & AD Classifier
        print("| FL Training Stage 1 Done|")

    elif args.FL_STAGE == 2:
        print("| Start FL Training Stage 2|")
        args.STAGE = 2
        stage2_training(args, train_dataset, test_dataset)                      # Train Toggling Network
        print("| FL Training Stage 2 Done|")
else:
    extract_emb(args, train_dataset, test_dataset)

print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))


FileNotFoundError: [Errno 2] No such file or directory: 'None/meta-data/test_dic.npy'

In [4]:
import pickle
import whisper
from tqdm import tqdm
import os
os.environ['DACS_codeRoot'] = '/home/FedASR/dacs/'
os.environ['DACS_dataRoot'] = '/mnt/Internal/FedASR/Data/ADReSS-IS2020-data/'
os.environ["CUDA_VISIBLE_DEVICES"] = "0,2,3"
from utils import csv2dataset, get_dataset, average_weights, exp_details


import argparse


def args_parser():
    parser = argparse.ArgumentParser()

    # federated arguments (Notation for the arguments followed from paper)
    parser.add_argument('--epochs', type=int, default=10,
                        help="number of rounds of training")
    parser.add_argument('--num_users', type=int, default=100,
                        help="number of users: K")
    parser.add_argument('--frac', type=float, default=0.1,
                        help='the fraction of clients: C')
    parser.add_argument('--local_ep', type=int, default=10,
                        help="the number of local epochs: E")
    parser.add_argument('--train_batch_size', type=int, default=6, help="")
    parser.add_argument('--eval_batch_size', type=int, default=8, help="")
    # model arguments
    parser.add_argument('--model', type=str, default='data2vec', help='model name')
    # other arguments
    parser.add_argument('--dataset', type=str, default='adress', help="name \
                        of dataset")
    parser.add_argument('--gpu', default=None, help="To use cuda, set \
                        to a specific GPU ID. Default set to use CPU.")
    # additional arguments
    parser.add_argument('--pretrain_name', type=str, default='facebook/data2vec-audio-large-960h', help="str used to load pretrain model")
    parser.add_argument('-lam', '--LAMBDA', type=float, default=0.5, help="Lambda for GRL")
    parser.add_argument('-st', '--STAGE', type=int, default=1, help="Current training stage")
    parser.add_argument('-fl_st', '--FL_STAGE', type=int, default=1, help="Current FL training stage")
    parser.add_argument('-GRL', '--GRL', action='store_true', default=False, help="True: GRL")
    parser.add_argument('-model_in', '--model_in_path', type=str, default="./saves/wav2vec2-base-960h_GRL_0.5/checkpoint-14010/", help="Where the global model is saved")
    parser.add_argument('-model_out', '--model_out_path', type=str, default="./saves/wav2vec2-base-960h_linear_GRL", help="Where to save the model")
    parser.add_argument('-log', '--log_path', type=str, default="wav2vec2-base-960h_linear_GRL.txt", help="name for the txt file")
    parser.add_argument('-csv', '--csv_path', type=str, default="wav2vec2-base-960h_GRL_0.5", help="name for the csv file")
    # 2023/01/08: loss type
    parser.add_argument('-ad_loss', '--AD_loss', type=str, default="cel", help="loss to use for AD classifier")
    # 2023/01/18: ckpt
    parser.add_argument('-ckpt', '--checkpoint', type=str, default=None, help="path to checkpoint")
    # 2023/02/13: TOGGLE_RATIO
    parser.add_argument('-toggle_rt', '--TOGGLE_RATIO', type=float, default=0, help="To toggle more or less")
    # 2023/02/15: GS_TAU, loss weight
    parser.add_argument('-gs_tau', '--GS_TAU', type=float, default=1, help="Tau for gumbel_softmax")
    parser.add_argument('-w_loss', '--W_LOSS', type=float, default=None, nargs='+', help="weight for HC and AD")
    # 2023/04/20
    parser.add_argument('-EXTRACT', '--EXTRACT', action='store_true', default=False, help="True: extract embs")
    parser.add_argument('-client_id', '--client_id', type=str, default="public", help="client_id: public, 0, or 1")
    # 2023/04/24
    parser.add_argument('--global_ep', type=int, default=30, help="number for global model")
    parser.add_argument('--GPU_batchsize', type=str, default=None, help="如果cpu滿了就用GPU")
    # 2023/05/18
    parser.add_argument('-sl', '--supervised_level', type=float, default=1, help="0=fully unsupervised; 0.5=unsupervised+supervised; 1=fully supervised")
    parser.add_argument('--num_lms', type=int, default=5, help="")
    # 2023/5/200
    parser.add_argument('--eval_steps', type=int, default=1000, help="")
    args = parser.parse_args(args=[])
    return args

args= args_parser()
class TeacherStudentLearning:
    def __init__(self, loadpath=None, savepath=None, load_mdl='large-v2'):
        self.transcript = []
        self.loadpath = loadpath
        self.savepath = savepath
        self.DACS_dataRoot = '/mnt/Internal/FedASR/Data/ADReSSo21/diagnosis/train'
        out_root='/mnt/Internal/FedASR/Data/ADReSSo21/diagnosis'
        out_dirname='transcript_whisper'
        out_filename='train.csv'
        self.out_file=f"{out_root}/{out_dirname}/{out_filename}"
        self.model = whisper.load_model(load_mdl)
    
    def add_transcript_to_dataset(self, dataset, transcript_in):
        dataset = dataset.add_column("text", transcript_in)
        return dataset
    
    def save_transcript(self, dataset,outFile):
        df = pd.DataFrame(dataset)
        df.to_csv(outFile, index=False)
    def load_transcript(self, in_file):
        df = pd.read_csv(in_file)
        dataset = Dataset.from_pandas(df)
        return dataset
    def transcribe(self, dataset):
        transcript=[]
        for i,batch in tqdm(enumerate(dataset)):
            singleFile=batch['path']
            file=f'{self.DACS_dataRoot}/clips/{singleFile}'
            result = self.model.transcribe(file, language="en")
            pred_text=result['text'].upper().strip()
            print(pred_text)
            transcript.append(pred_text)
        return transcript
    def transcribe_n_Merge(self, dataset):
        transcript = self.transcribe(dataset)
        ds=self.add_transcript_to_dataset(self, dataset, transcript)
        return ds
    def FilterAvailAudios(self, dataset):
        dataset_enoughLen = dataset.filter(lambda example: len(example['array']) >= 1600, input_columns=['array'])
        dataset_enoughLen_enoughtext = dataset.filter(lambda example: len(example['text']) >= 0, input_columns=['text'])
        return dataset_enoughLen_enoughtext


args.dataset = "adresso"                                                        # get unsupervised dataset (adresso)
train_dataset_unsupervised, _ = get_dataset(args)                               # get dataset w.o. testing set

def split_list(lst, num_parts):
    total_samples = len(lst)
    samples_per_part = total_samples // num_parts

    # 切割成N等分的索引范围
    indices = [(i * samples_per_part, (i + 1) * samples_per_part) for i in range(num_parts)]

    # 处理最后一个部分，以处理无法整除的情况
    last_index = num_parts * samples_per_part
    if last_index < total_samples:
        indices.append((last_index, total_samples))

    # 建立切割的子列表
    split_lists = []
    for start, end in indices:
        split_lists.append(lst[start:end])

    return split_lists

# 设置要切割的等分数量
num_splits = 40
slices = train_dataset_addresso.train_test_split(test_size=1/40)
aaa=cccc
# 将列表切割成N等分
split_datasets = split_list(train_dataset_unsupervised, num_splits)

ds_new=TSL.transcribe_n_Merge(self, dataset)



# 創建一個 TranscriptDataset 物件
TSL = TeacherStudentLearning()
# transcript=TSL.transcribe(train_dataset_unsupervised)

pool = multiprocessing.Pool(processes=num_splits)
try:
    final_result = pool.starmap_async(
        TSL.transcribe, [(dataset)
                    for dataset in split_datasets])
except Exception as e:
    print(f"An error occurred while running local_model.update_weights(): {str(e)}")
finally:
    final_result.wait()
    results = final_result.get()

for idx in range(len(results)):                                             # for each participated clients
    w, loss = results[idx]                                                  # function client_train returns w *& loss
    if args.STAGE == 0:                                                     # train ASR
        local_weights_en.append(copy.deepcopy(w[0]))                        # save encoder weight for this client
        local_weights_de.append(copy.deepcopy(w[1]))                        # save decoder weight for this client
    else:                                                                   # train AD classifier or toggling network
        local_weights.append(copy.deepcopy(w))
    local_losses.append(loss)


train_dataset_addresso=TSL.add_transcript_to_dataset(train_dataset_unsupervised, transcript)
out_root='/mnt/Internal/FedASR/Data/ADReSSo21/diagnosis'
out_dirname='transcript_whisper'
out_filename='train.csv'
out_file=f"{out_root}/{out_dirname}/{out_filename}"
TSL.save_transcript(train_dataset_addresso, out_file)
in_file=out_file
train_dataset_addresso_loaded=TSL.load_transcript(in_file)

/home/FedASR/.conda/envs/Flower-speechbrain/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading cached processed dataset at /home/FedASR/dacs/federated/src/dataset/ADReSSo/train_ADReSSo/cache-5f84b687fbc2e2ed_*_of_00010.arrow


Load data from local...


NameError: name 'train_dataset_addresso' is not defined

In [5]:
args.dataset = "adresso"                                                        # get unsupervised dataset (adresso)
train_dataset_unsupervised, _ = get_dataset(args)                               # get dataset w.o. testing set
print(len(train_dataset_unsupervised[0]['array']))


Loading cached processed dataset at /home/FedASR/dacs/federated/src/dataset/ADReSSo/train_ADReSSo/cache-5f84b687fbc2e2ed_*_of_00010.arrow


Load data from local...
124113
